In [7]:
import pandas as pd
import numpy as np
import cv2

import face_rec

In [2]:
face_rec.r.ping()

True

#### Step1: Extract Data from Redis

In [11]:
table = 'academy:register'
retrieve_dict = face_rec.r.hgetall(table)
retrieve_series = pd.Series(retrieve_dict)
retrieve_series = retrieve_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))
index = retrieve_series.index
index = list(map(lambda x: x.decode(), index))
retrieve_series.index = index
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['name_role', 'facial_features']
retrieve_df[['Name', 'Role']] = retrieve_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)
retrieve_df

,name_role,facial_features,Name,Role
0,Chris Evans@Student,"[0.72012275, -1.4683539, -0.77757514, -1.46026...",Chris Evans,Student
1,Barack Obama@Teacher,"[0.948603, -0.558349, 0.09484784, -1.2796344, ...",Barack Obama,Teacher
2,Angelina Jolie@Student,"[0.43222713, 1.074822, -0.12857957, -1.8332443...",Angelina Jolie,Student
3,Morgan Freeman@Teacher,"[0.4857939, -0.61931694, -0.4072946, 1.0686711...",Morgan Freeman,Teacher
4,Scarlett Johansson@Student,"[0.1913062, -0.48871604, -1.6904205, 0.4424578...",Scarlett Johansson,Student
5,Momo@Teacher,"[-0.3169872, -1.3053955, -0.7045331, -0.363322...",Momo,Teacher


#### Step3: Get Real Time Prediction

In [14]:
cap = cv2.VideoCapture(0)

while True:
    success, frame = cap.read()
    if not success:
        break
    
    result = face_rec.face_prediction(frame, retrieve_df, 'facial_features', None)
    # cv2.imshow('Frame', frame)
    cv2.imshow('Prediction', result)
    
    if cv2.waitKey(1) == 27: # if press esc button
        break


cv2.destroyAllWindows()
cap.release()
cv2.waitKey(1)

-1